In [ ]:
import pandas as pd
import requests
import json
import re
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
class openai:
    def __init__(self) -> None:
        pass

    def consumer_details(self):
        try:
            """get consumer details"""
            url = os.getenv("login_api") 
            response = requests.get(url )

            if response.status_code == 200:
                print("consumer details success")
                return response.json() 
        except Exception as e:
            print("consumer details error:", e)

    def get_population(self,year):
        """get usa population"""
        url = "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
        response = requests.get(url )
    
        if response.status_code == 200:
            # print("Success:", response.json())
            print("usa popluation data success")
            return response.json()
        else:
            print("Error in api:", response.status_code, response.text)
    
    def get_current_weather(self, location, unit="celcius"):
        """Get the current weather in a given location"""
        if "tokyo" in location.lower():
            return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
        elif "san francisco" in location.lower():
            return json.dumps({"location": "San Francisco", "temperature": "32", "unit": unit})
        elif "paris" in location.lower():
            return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
        else:
            return json.dumps({"location": location, "temperature": "unknown"})
        
    def format_currency_as_rupees(self,response):
        # Use regex to find and replace dollar amounts with rupees
        return re.sub(r'\$\s*([\d,]+(?:\.\d+)?)', r'₹\1', response)

In [ ]:
data = openai()
output = data.consumer_details()
output.keys()

In [ ]:
keys_to_fetch = ['consumer_name',"consumer_mobile_no","consumer_email_id","balance_amount",'last_recharge_time', 'last_coupon_number', 'last_coupon_amount','dg_reading', 'grid_reading',"grid_sanctioned_load",]
{key: output['resource'][key] for key in keys_to_fetch}

In [ ]:
fetched_data = {key: output['resource'][key] for key in keys_to_fetch}


In [ ]:
fetched_data

In [ ]:
output['resource'].keys()

In [ ]:
output['resource']

In [ ]:
! pip list

In [ ]:
import time
l = [100,3,93,81,74,54,65,51,43,35,28,16,75,48,20,17,55,99,62]


In [ ]:
# bubble sort
start_time = time.time()
print(start_time)
# l= [100,9,81,3,26]
for y in range(len(l)):
    for i in range(len(l)-1):
        if l[i]>l[i+1]:
            l[i],l[i+1]=l[i+1],l[i]
        print(i,l)
end_time = time.time()
print(end_time)
print(end_time-start_time)

In [ ]:
# selection sort
# l= [100,9,81,3,26]
start_time = time.time()
print(start_time)
print(l)
for y in range (len(l)):
    for i in range (y+1 , len(l)):
        # print("l:",l)
        if l[y]>l[i]:
            l[y],l[i]=l[i],l[y]
        print(y,i,l)

end_time = time.time()
print(end_time)
print(end_time-start_time)

In [60]:
def fact(n):
    output = 1
    if n==1:
        return output
    else:
        output = n*(n-1)
        fact(n-1)

fact(2)